<a href="https://colab.research.google.com/github/jackysiupuichung/adme_property_prediction/blob/main/models/ADME_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from numpy import genfromtxt

df = pd.read_csv('/content/gdrive/MyDrive/adme_benchmarking/dataframe.csv')

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df.Drug

0              CC(=O)N(c1ccc2oc(=O)sc2c1)S(=O)(=O)c1cccs1
1         COc1ccccc1C(c1nnnn1C(C)(C)C)N1CCN(Cc2ccncc2)CC1
2           CC(=O)Nc1cccc(NC(=O)C2CCCN2C(=O)Nc2ccccc2C)c1
3       CCC(c1nnnn1CC1CCCO1)N(CCN1CCOCC1)Cc1cc2cc(C)cc...
4       Br.N=c1n(CCN2CCOCC2)c2ccccc2n1CC(=O)c1ccc(Cl)c...
                              ...                        
9927             C=CC1=C[C@@H](O)[C@@H]2O[C@@H]2C12OCCCO2
9928    O=C1[C@H]2CC=C3[C@@H]([C@H](O)[C@H]4O[C@@H]4C3...
9929    CCN1C(=O)[C@H]2CC=C3[C@@H]([C@H](O)[C@H]4O[C@@...
9930    C[C@H](c1ccccc1)N1C(=O)[C@@H]2[C@@H](CC[C@@H]3...
9931                          COC(=O)NC/C=C\c1nc(CCCO)co1
Name: Drug, Length: 9932, dtype: object

In [ ]:
dataset = df[['Drug', 'CYP2D6']]

In [ ]:
import collections

collections.Counter(dataset['CYP2D6'].to_list())

Counter({0: 7998, 1: 1934})

In [ ]:
from sklearn.model_selection import train_test_split

train_dataset, test_dataset = train_test_split(dataset,test_size=0.4, stratify = dataset['CYP2D6'], random_state=123)

test_dataset, val_dataset = train_test_split(test_dataset,test_size=0.5, random_state=123)

print(train_dataset.shape)
print(collections.Counter(train_dataset['CYP2D6'].to_list()))
print(test_dataset.shape)
print(collections.Counter(test_dataset['CYP2D6'].to_list()))
print(val_dataset.shape)
print(collections.Counter(val_dataset['CYP2D6'].to_list()))


(5959, 2)
Counter({0: 4799, 1: 1160})
(1986, 2)
Counter({0: 1614, 1: 372})
(1987, 2)
Counter({0: 1585, 1: 402})


In [ ]:
from simpletransformers.classification import ClassificationModel
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [67]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model = ClassificationModel('roberta', 'seyonec/PubChem10M_SMILES_BPE_396_250', args={'train_batch_size':32, 'eval_batch_size':32, 'evaluate_each_epoch': True, 'evaluate_during_training_verbose': True, 'no_save': True, 'num_train_epochs': 15, 'auto_weights': True})

Some weights of the model checkpoint at seyonec/PubChem10M_SMILES_BPE_396_250 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/PubChem10M_SMILES_BPE

In [ ]:
print(model.tokenizer)

PreTrainedTokenizerFast(name_or_path='seyonec/PubChem10M_SMILES_BPE_396_250', vocab_size=7924, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})


In [68]:
import sklearn

#evaluate the pre-trained model's capability in predicting for the task
# accuracy
result, model_outputs, wrong_predictions = model.eval_model(test_dataset, acc=sklearn.metrics.accuracy_score)
print(result)

# ROC-PRC
result, model_outputs, wrong_predictions = model.eval_model(test_dataset, acc=sklearn.metrics.average_precision_score)
print(result)

/usr/local/lib/python3.8/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1986 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/63 [00:00<?, ?it/s]

{'mcc': -0.06761603721959607, 'tp': 114, 'tn': 984, 'fp': 630, 'fn': 258, 'auroc': 0.45073933058853316, 'auprc': 0.1727908882788487, 'acc': 0.552870090634441, 'eval_loss': 0.7007703024243551}


/usr/local/lib/python3.8/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1986 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/63 [00:00<?, ?it/s]

{'mcc': -0.06761603721959607, 'tp': 114, 'tn': 984, 'fp': 630, 'fn': 258, 'auroc': 0.45073933058853316, 'auprc': 0.1727908882788487, 'acc': 0.17686566108440666, 'eval_loss': 0.7007703024243551}


In [ ]:
!mkdir /content/gdrive/MyDrive/adme_benchmarking/transformer

mkdir: cannot create directory ‘/content/gdrive/MyDrive/adme_benchmarking/transformer’: File exists


In [69]:
# Train the model
model.train_model(train_dataset, eval_df=val_dataset, output_dir='/content/gdrive/MyDrive/adme_benchmarking/transformer/adme_CYP2D6')

/usr/local/lib/python3.8/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/5959 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/187 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/187 [00:00<?, ?it/s]

...

Running Epoch 11 of 15:   0%|          | 0/187 [00:00<?, ?it/s]

Running Epoch 12 of 15:   0%|          | 0/187 [00:00<?, ?it/s]

Running Epoch 13 of 15:   0%|          | 0/187 [00:00<?, ?it/s]

Running Epoch 14 of 15:   0%|          | 0/187 [00:00<?, ?it/s]

(2805, 0.1060369475010396)

In [70]:
import sklearn

# accuracy
result, model_outputs, wrong_predictions = model.eval_model(test_dataset, acc=sklearn.metrics.accuracy_score)
print(result)

# ROC-PRC
result, model_outputs, wrong_predictions = model.eval_model(test_dataset, acc=sklearn.metrics.average_precision_score)
print(result)

/usr/local/lib/python3.8/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1986 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/63 [00:00<?, ?it/s]

{'mcc': 0.49725639224995954, 'tp': 190, 'tn': 1515, 'fp': 99, 'fn': 182, 'auroc': 0.8318968101690849, 'auprc': 0.6424546858577044, 'acc': 0.8585095669687814, 'eval_loss': 1.1185676162205045}


/usr/local/lib/python3.8/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1986 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/63 [00:00<?, ?it/s]

{'mcc': 0.49725639224995954, 'tp': 190, 'tn': 1515, 'fp': 99, 'fn': 182, 'auroc': 0.8318968101690849, 'auprc': 0.6424546858577044, 'acc': 0.4274304549752048, 'eval_loss': 1.1185676162205045}
